The primary task is to evaluate the performance of the GPT-3.5 Turbo model in answering multiple-choice questions. 
The code provided seems to be a part of this evaluation process. 
Specifically, it's aimed at assessing how well the model can answer questions by comparing its responses to the correct answers



In [1]:
import openai , requests , json 
import pandas as pd


In [17]:
import os

openai.api_key = "your-key"

In [2]:
df=pd.read_csv('llmtrain.csv')

In [3]:
df

,id,prompt,A,B,C,D,E,answer
0,0,Which of the following statements accurately d...,MOND is a theory that reduces the observed mis...,MOND is a theory that increases the discrepanc...,MOND is a theory that explains the missing bar...,MOND is a theory that reduces the discrepancy ...,MOND is a theory that eliminates the observed ...,D
1,1,Which of the following is an accurate definiti...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,A
2,2,Which of the following statements accurately d...,The triskeles symbol was reconstructed as a fe...,The triskeles symbol is a representation of th...,The triskeles symbol is a representation of a ...,The triskeles symbol represents three interloc...,The triskeles symbol is a representation of th...,A
3,3,What is the significance of regularization in ...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,C
4,4,Which of the following statements accurately d...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,D
...,...,...,...,...,...,...,...,...
195,195,What is the relation between the three moment ...,The three moment theorem expresses the relatio...,The three moment theorem is used to calculate ...,The three moment theorem describes the relatio...,The three moment theorem is used to calculate ...,The three moment theorem is used to derive the...,C
196,196,"What is the throttling process, and why is it ...",The throttling process is a steady flow of a f...,The throttling process is a steady adiabatic f...,The throttling process is a steady adiabatic f...,The throttling process is a steady flow of a f...,The throttling process is a steady adiabatic f...,B
197,197,What happens to excess base metal as a solutio...,"The excess base metal will often solidify, bec...",The excess base metal will often crystallize-o...,"The excess base metal will often dissolve, bec...","The excess base metal will often liquefy, beco...","The excess base metal will often evaporate, be...",B
198,198,"What is the relationship between mass, force, ...",Mass is a property that determines the weight ...,Mass is an inertial property that determines a...,Mass is an inertial property that determines a...,Mass is an inertial property that determines a...,Mass is a property that determines the size of...,D


In [5]:
row=df.iloc[0]

In [7]:
system_message = """Answer the follwing multiple-choice question by providing your top 3 guesses in order from 
most to least likely, using the following format: 'A C D' (just the letters separated by spaces)."""
user_message = f"""Question: {row['prompt']}. Answers: {' '.join([f'{l}: {row[l]}' for l in 'ABCDE'])}"""

In [9]:
system_message

"Answer the follwing multiple-choice question by providing your top 3 guesses in order from \nmost to least likely, using the following format: 'A C D' (just the letters separated by spaces)."

In [8]:
user_message

'Question: Which of the following statements accurately describes the impact of Modified Newtonian Dynamics (MOND) on the observed "missing baryonic mass" discrepancy in galaxy clusters?. Answers: A: MOND is a theory that reduces the observed missing baryonic mass in galaxy clusters by postulating the existence of a new form of matter called "fuzzy dark matter." B: MOND is a theory that increases the discrepancy between the observed missing baryonic mass in galaxy clusters and the measured velocity dispersions from a factor of around 10 to a factor of about 20. C: MOND is a theory that explains the missing baryonic mass in galaxy clusters that was previously considered dark matter by demonstrating that the mass is in the form of neutrinos and axions. D: MOND is a theory that reduces the discrepancy between the observed missing baryonic mass in galaxy clusters and the measured velocity dispersions from a factor of around 10 to a factor of about 2. E: MOND is a theory that eliminates the

In [18]:
response=openai.ChatCompletion.create(
    model='gpt-3.5-turbo',
    messages=[
      {"role": "system", "content": system_message},
        {"role": "user", "content": user_message}
    ]
)

In [19]:
response

<OpenAIObject chat.completion id=chatcmpl-89Ywv0cmG6Geo3RznXOMrfTTpgmXY at 0x1f220124f90> JSON: {
  "id": "chatcmpl-89Ywv0cmG6Geo3RznXOMrfTTpgmXY",
  "object": "chat.completion",
  "created": 1697289989,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "A B D"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 299,
    "completion_tokens": 3,
    "total_tokens": 302
  }
}

In [23]:
response['choices'][0]['message']['content']

'A B D'

In [29]:
def generate_answer(row):
    # System prompt
    system_message = f"""Answer the follwing multiple-choice question by providing your top 3 guesses along with your reasoning."""

    # Set the user message with the specific context
    user_message = user_message = f"""Question: {row['prompt']}. Answers: {' '.join([l+': '+row[l] for l in 'ABCDE'])}"""

    # Define the function(s) the model will be able to use (in this case, only one)
    functions = [
        {
            "name": "answer_question",
            "description": "Answers the provided question",
            "parameters": {
                "type": "object",
                "properties": {
                    "reasoning": {
                    "type": "string",
                    "description": "Reasining for what the answer could be. Keep it short."
                    },
                    "answers": {
                    "type": "array",
                    "items": {
                        "type": "string",
                        "enum": ["A", "B", "C", "D", "E"],
                    },
                    "description": "Your top 3 guesses, from most to least likely. e.g. ['A', 'D', 'C']"
                    }
                },
                "required": ["reasoning", "answers"],
            },
        }
    ]

    # Call the chat completions API with the function calling parameters
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": user_message}
        ],
        functions=functions, # Add the function
        function_call={"name": "answer_question"} # Force the function call
    )

    # Extract the function call arguments
    args = json.loads(response['choices'][0]['message']['function_call']['arguments'])

    return args

In [30]:
generate_answer(df.iloc[0])

{'reasoning': 'I am not familiar with the impact of Modified Newtonian Dynamics (MOND) on the observed missing baryonic mass in galaxy clusters. However, based on the given statements, I can provide my top 3 guesses.',
 'answers': ['A', 'B', 'D']}

In [31]:
score = 0
for idx, row in df[:100].iterrows():
    a = generate_answer(row)
    answers = a['answers'][:3] # No need for string splitting :)
    correct_answer = row['answer']
    for i, guess in enumerate(answers):
        if guess == correct_answer:
            score += 1/(i+1) # 1 for first guess, 1/2 for second and 1/3 for third
            
print("Average Score:", score/100)

Average Score: 0.7266666666666666
